In [1]:
% pip install git+https://github.com/albumentations-team/albumentations


  Cloning https://github.com/albumentations-team/albumentations to c:\users\admin\appdata\local\temp\pip-req-build-oxwh9qjk
  Resolved https://github.com/albumentations-team/albumentations to commit edb09e36cfe68d159be2167d6b02fb44d995d5a1
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/albumentations-team/albumentations 'C:\Users\Admin\AppData\Local\Temp\pip-req-build-oxwh9qjk'


In [2]:
import torch
import torch.nn as nn
import os
import cv2
import albumentations as A
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
from torchvision import models
from torchvision import transforms
from torch.utils.data import DataLoader
from albumentations.pytorch import ToTensorV2
import random
import copy

In [4]:
if(os.path.split(os.getcwd())[1] == "plain-augmentation"):
    os.chdir("..")
print("Current Working Directory: {}".format(os.path.split(os.getcwd())[1]))

cuda=False

Current Working Directory: progettoVIPM


In [ ]:
random.seed(42)
train_transform = A.Compose(
    [
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RandomCrop(height=128, width=128, p=0.5),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),        
        A.RandomToneCurve(scale=0.7, p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),#as required by All pre-trained models of torch
        ToTensorV2(),
    ]
)


from utils.loadersAndEnums import datasets
from utils.loadersAndEnums import networks
from utils.loadersAndEnums import ImageDataset
from utils.visualizing import visualize_some_image_in_loader

datasetholder = ImageDataset(dataset=datasets.TRAINING_LABELED, network_input_size=256, cuda=False, transform=train_transform)
loader = DataLoader(dataset=datasetholder, shuffle=False, batch_size=1)


visualize_some_image_in_loader(loader, 4)
